# Assigment 2

<hr style="border:2px solid gray">
## Ex1

### GATK naive Bayesian method

The basic formula we use for all types of variation under consideration (SNPs, insertions and deletions) is:

$$ P(G|D) = \frac{ P(G) P(D|G) }{ \sum_{i} P(G_i) P(D|G_i) } $$

If that is meaningless to you, please don't freak out -- we're going to break it down and go through all the components one by one. First of all, the term on the left:

$$ P(G|D) $$

is the quantity we are trying to calculate for each possible genotype: the conditional probability of the genotype G given the observed data D.

Now let's break down the term on the right:

$$ \frac{ P(G) P(D|G) }{ \sum_{i} P(G_i) P(D|G_i) } $$

We can ignore the denominator (bottom of the fraction) because it ends up being the same for all the genotypes, and the point of calculating this likelihood is to determine the most likely genotype. The important part is the numerator (top of the fraction):

$$ P(G) P(D|G) $$

which is composed of two things: the prior probability of the genotype and the conditional probability of the data given the genotype.

The first one is the easiest to understand. The prior probability of the genotype G:

$$ P(G) $$

represents how probably we expect to see this genotype based on previous observations, studies of the population, and so on. By default, the GATK tools use a flat prior (always the same value) but you can input your own set of priors if you have information about the frequency of certain genotypes in the population you're studying.

The second one is a little trickier to understand if you're not familiar with Bayesian statistics. It is called the conditional probability of the data given the genotype, but what does that mean? Assuming that the genotype G is the true genotype,

$$ P(D|G) $$

is the probability of observing the sequence data that we have in hand. That is, how likely would we be to pull out a read with a particular sequence from an individual that has this particular genotype? We don't have that number yet, so this requires a little more calculation, using the following formula:

$$ P(D|G) = \prod{j} \left( \frac{P(D_j | H_1)}{2} + \frac{P(D_j | H_2)}{2} \right) $$

You'll notice that this is where the diploid assumption comes into play, since here we decomposed the genotype G into:

$$ G = H_1H_2 $$

which allows for exactly two possible haplotypes. In future versions we'll have a generalized form of this that will allow for any number of haplotypes.

Now, back to our calculation, what's left to figure out is this:

$$ P(D_j|H_n) $$

which as it turns out is the conditional probability of the data given a particular haplotype (or specifically, a particular allele), aggregated over all supporting reads. Conveniently, that is exactly what we calculated in Step 3 of the HaplotypeCaller process, when we used the PairHMM to produce the likelihoods of each read against each haplotype, and then marginalized them to find the likelihoods of each read for each allele under consideration. So all we have to do at this point is plug the values from that table into the equation above, and we can work our way back up to obtain:

$$ P(G|D) $$

for the genotype G.

In [14]:
data <- read.table('input.txt')

summary(data)
head(data,5)

       V1           V2                V3                  V4        
 Min.   :20   Min.   : 4369726   Length:560         Min.   : 40.00  
 1st Qu.:20   1st Qu.:29653244   Class :character   1st Qu.: 46.00  
 Median :20   Median :29829452   Mode  :character   Median : 57.00  
 Mean   :20   Mean   :32412121                      Mean   : 64.04  
 3rd Qu.:20   3rd Qu.:29829727                      3rd Qu.: 76.00  
 Max.   :20   Max.   :47132254                      Max.   :158.00  
      V5                 V6           
 Length:560         Length:560        
 Class :character   Class :character  
 Mode  :character   Mode  :character  
                                      
                                      
                                      

,V1,V2,V3,V4,V5,V6
,<int>,<int>,<chr>,<int>,<chr>,<chr>
1,20,4369726,G,113,"G,G,G,G,G,G,G,G,G,G,C,G,G,G,G,G,G,G,C,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G","27,32,33,34,31,32,34,33,30,34,31,33,32,33,33,33,33,30,31,29,32,33,29,33,33,33,33,32,32,34,33,33,34,32,33,34,32,33,33,33,33,33,29,32,32,33,31,33,33,33,32,32,33,33,33,32,30,33,31,33,33,33,33,33,34,34,31,32,32,32,32,32,32,32,33,33,31,31,32,32,31,33,33,32,31,31,31,32,33,33,33,31,33,31,31,33,31,33,31,29,32,32,32,33,32,33,33,31,32,28,32,31,31"
2,20,4369728,T,113,"T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,G,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T","33,31,31,30,31,31,30,31,32,31,30,27,31,30,31,30,29,31,29,30,32,29,31,31,32,32,29,30,31,31,32,31,32,31,31,29,32,31,32,29,32,29,32,32,32,29,32,31,32,32,31,32,32,32,30,31,32,25,32,28,32,32,32,32,32,29,30,28,30,30,32,30,31,31,32,30,30,30,30,30,31,32,30,30,30,30,31,32,32,32,30,31,30,30,32,28,32,30,30,31,30,31,31,31,26,32,32,27,25,31,30,30,29"
3,20,4369731,A,113,"A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,G,A,A,A,A,A","32,34,33,32,32,32,32,33,33,32,31,34,30,34,34,30,32,31,32,33,30,33,34,34,33,31,31,31,32,33,33,33,33,31,31,31,33,33,33,33,30,33,33,33,30,33,33,33,33,33,33,33,33,32,32,33,27,33,32,33,33,33,33,33,31,32,31,32,32,33,29,33,33,33,32,32,32,33,30,33,33,29,32,32,32,32,33,33,33,32,27,32,32,33,32,33,32,30,33,32,33,33,32,33,33,33,33,25,33,32,32,32,31"
4,20,4369733,T,110,"T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T","33,33,32,33,33,29,33,29,27,30,33,29,32,33,33,32,33,29,33,31,33,33,32,28,32,32,33,32,33,33,33,28,32,32,32,32,33,21,32,33,33,31,33,32,32,32,33,33,32,33,32,29,33,30,32,25,33,32,33,32,32,28,31,32,31,31,33,33,32,18,32,31,31,31,33,31,32,33,31,31,30,31,33,33,32,33,31,31,31,31,32,31,32,31,31,32,31,32,31,33,30,32,33,32,31,32,31,31,32,31"
5,20,4369734,C,111,"C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,G,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C","32,32,32,30,26,31,33,33,27,29,33,31,33,33,33,29,32,33,30,33,33,33,32,31,30,33,31,32,33,32,33,32,30,31,33,32,33,32,32,32,32,32,31,32,32,32,32,32,32,32,32,32,31,32,30,32,30,32,32,32,32,32,31,31,32,31,31,32,31,32,32,32,31,31,31,32,31,32,32,31,31,31,31,32,33,33,33,31,31,31,31,32,31,32,31,31,32,31,32,28,32,32,32,32,32,31,32,31,31,31,32"


In [16]:
# Taken from lectures
compute_base_probability <- function(base, qual, genotype) {
  e <- 10^(-qual/10)  # probability that the base call is incorrect
  
  #print(base == genotype)
  if (base != genotype) {
    return(e/3)
  } else {
    return(1 - e)  
  }
}

# Function to compute probabilities based on Phred scores
compute_p <- function(bases, quals, genotypes,priors=NULL) {
  # Assuming uniform prior probabilities for all genotypes
  if (is.null(priors)) {
    priors <- rep(1/length(genotypes), length(genotypes))
  }
  
  # Initialize an empty vector to store the unnormalized posteriors
  ps <- numeric(length(genotypes))
  
  # Iterate over each genotype
  for (g_idx in 1:length(genotypes)) {
    # Placeholder for unnormalized posterior
    unnormalized_posterior <- 1
    #print(length(genotypes))
    # Iterate over each base in the pileup
    for (b_idx in 1:length(bases)) {
      # Extract the base and quality from the current read
      base <- toupper(bases[b_idx])
      qual <- quals[b_idx]
      # Print for debugging
      #print(paste("Genotype:", genotypes[g_idx], "Base:", base, "Index:", b_idx))

      # Use the separate function to compute the probability of the base
      #print(genotypes[g_idx])
      p_base_given_genotype1 <- 0.5*compute_base_probability(base, qual, substr(genotypes[g_idx], 2, 2))
      p_base_given_genotype2 <- 0.5*compute_base_probability(base, qual, substr(genotypes[g_idx], 1, 1))
        
      p_base_given_genotype=p_base_given_genotype1+p_base_given_genotype2
      # Multiply the probability with the prior for the genotype
      #unnormalized_posterior <- unnormalized_posterior * p_base_given_genotype
      #print(unnormalized_posterior)
      unnormalized_posterior <- unnormalized_posterior * p_base_given_genotype * priors[g_idx]
      #print(paste("Genotype:", genotypes[g_idx], "Base:", base, "Probability:", p_base_given_genotype, "Unnormalized Posterior:", unnormalized_posterior))
      #unnormalized_posterior <- unnormalized_posterior * p_base_given_genotype * priors[g_idx]
        
      #if (b_idx >= 3) {
    #break
  #}
}
    
    # Store the unnormalized posterior for the current genotype
    ps[g_idx] <- unnormalized_posterior
  }
  
  
  return(ps)
}

In [24]:

# Initialize a vector to store the selected genotypes
selected_genotypes <- character(nrow(data))
# Your loop
for (i in 1:nrow(data)) {
  bases <- unlist(strsplit(as.character(data[i, 5]), split = ","))
  quals <- as.numeric(unlist(strsplit(as.character(data[i, 6]), split = ",")))
  genotypes <- c("GG", "GC", "GA", "GT", "CC", "CA", "CT", "AA", "AT", "TT")
  ps <- compute_p(bases, quals, genotypes)
  
  # Select the genotype with the maximum unnormalized posterior
  selected_genotypes[i] <-  genotypes[which.max(ps)]
}

# Create a table of selected genotypes
genotype_table <- table(selected_genotypes)

# Print the table
print(genotype_table)

selected_genotypes
 AA  AT  CA  CC  CT  GA  GC  GG  GT  TT 
124  10  16  68  33  34  25 146  11  93 


[1] "GG" "TT" "AA" "TT" "CC" "CC" "GG" "AA" "TT" "AA" "TT" "GG" "AA" "GG" "TT"
 [16] "GG" "TT" "TT" "CC" "GG" "GG" "GG" "GG" "CC" "GG" "AA" "AA" "CC" "TT" "TT"
 [31] "GG" "AA" "GG" "CC" "TT" "GG" "CT" "CT" "CA" "TT" "AA" "AA" "GG" "CC" "CC"
 [46] "AA" "AT" "CC" "TT" "CT" "GC" "GC" "AA" "GA" "CC" "GG" "TT" "GG" "GA" "CA"
 [61] "GA" "GT" "CC" "GG" "GG" "GG" "GA" "TT" "CT" "GG" "TT" "CA" "GA" "AT" "GG"
 [76] "CC" "AA" "GG" "TT" "AA" "GG" "GA" "CA" "CC" "GG" "GA" "CT" "GG" "GC" "TT"
 [91] "GG" "TT" "GA" "TT" "CT" "GG" "GG" "CC" "CC" "TT" "AT" "CT" "CT" "GA" "TT"
[106] "CT" "CC" "GT" "CA" "TT" "GA" "CT" "GG" "GA" "CT" "TT" "GT" "CT" "CT" "GA"
[121] "TT" "CC" "CC" "CC" "GG" "CT" "GG" "AT" "CC" "CC" "TT" "GG" "GG" "CT" "GC"
[136] "CT" "GA" "CA" "CC" "AA" "GA" "GT" "CT" "CA" "GC" "CT" "CT" "GA" "GA" "GA"
[151] "GA" "GA" "GG" "CC" "GG" "TT" "GG" "GT" "AA" "AA" "TT" "GG" "GG" "CA" "TT"
[166] "GG" "GG" "AA" "TT" "GC" "AA" "AA" "GG" "GG" "AA" "TT" "GG" "GG" "AA" "TT"
[181] "GG" "GG" "AA" "TT" "GG" "GG" "AA" "AA" "TT" "AA" "TT" "GG" "AA" "AA" "GC"
[196] "TT" "GG" "CT" "GG" "GG" "AA" "GG" "GG" "TT" "GG" "TT" "GG" "AA" "TT" "GG"
[211] "AT" "AA" "TT" "GG" "AA" "AA" "TT" "GG" "TT" "GG" "GG" "GG" "AT" "CT" "GG"
[226] "AA" "AA" "AA" "GG" "TT" "GG" "GT" "AA" "CT" "GG" "GG" "AA" "AT" "GA" "GC"
[241] "AA" "AA" "CT" "GA" "GT" "CA" "GC" "AT" "GT" "AA" "GG" "AA" "AA" "CT" "CC"
[256] "GA" "TT" "GG" "CC" "GG" "AA" "GG" "GC" "AA" "AA" "GG" "AA" "AA" "GC" "AA"
[271] "AA" "GG" "TT" "GG" "GG" "TT" "GC" "AA" "AA" "GG" "GG" "AA" "AA" "GG" "AA"
[286] "AA" "TT" "GG" "GG" "AA" "AA" "TT" "GG" "GG" "AA" "TT" "GG" "GG" "AA" "AA"
[301] "GG" "AA" "AA" "TT" "GC" "AA" "AA" "TT" "GG" "GG" "AA" "AA" "GC" "AA" "GG"
[316] "GG" "GG" "AA" "GG" "AA" "TT" "CC" "AA" "AA" "TT" "GG" "AA" "TT" "GG" "AA"
[331] "TT" "GG" "TT" "GG" "GG" "GG" "AA" "AA" "GG" "GC" "AA" "TT" "GG" "AA" "TT"
[346] "GG" "GG" "TT" "GA" "GG" "TT" "AA" "GC" "AA" "AA" "GG" "AA" "GG" "GG" "AA"
[361] "AA" "TT" "GG" "CC" "GA" "GG" "CA" "TT" "GG" "GA" "GA" "AA" "CA" "TT" "GC"
[376] "GG" "TT" "GG" "AT" "TT" "GC" "AA" "GG" "AA" "AA" "AA" "TT" "GG" "AA" "TT"
[391] "GA" "AA" "TT" "GG" "GG" "AA" "GG" "GG" "AA" "GC" "GA" "AA" "TT" "GG" "GG"
[406] "AA" "AA" "AA" "GG" "GG" "AA" "AA" "GG" "AA" "CA" "TT" "GG" "GG" "AA" "GG"
[421] "GG" "AA" "GC" "AA" "TT" "GG" "GG" "AA" "TT" "GG" "AA" "GA" "TT" "GG" "GG"
[436] "AA" "TT" "GG" "TT" "GA" "GG" "TT" "TT" "TT" "AA" "AA" "TT" "AA" "AA" "CC"
[451] "CC" "CC" "GG" "CC" "CC" "CC" "CA" "CT" "CC" "AA" "TT" "CC" "CC" "GG" "TT"
[466] "CC" "CC" "CT" "TT" "CC" "CC" "GC" "GG" "CA" "TT" "CT" "CC" "CT" "GA" "TT"
[481] "CC" "TT" "AA" "AA" "AA" "AA" "AA" "CC" "TT" "TT" "CC" "CT" "GG" "CC" "CT"
[496] "GG" "AA" "CC" "CC" "GG" "GG" "CC" "GT" "CC" "CT" "CC" "GG" "CC" "CC" "GC"
[511] "TT" "CC" "AA" "GA" "CC" "TT" "GG" "CC" "CC" "CC" "CC" "GG" "GT" "GT" "CC"
[526] "TT" "CT" "AA" "CC" "GG" "TT" "AA" "GC" "AA" "GG" "AA" "CC" "CC" "CC" "GC"
[541] "TT" "AT" "CC" "GG" "TT" "GA" "CC" "CC" "CC" "CT" "GC" "AA" "AA" "CA" "GA"
[556] "CA" "AA" "TT" "GG" "CC"

In [48]:

# priors for "ALL", "EUR", and "FIN"
population_priors_ALL <- c(0.308, 0.294, 0.385, 0.011, 0.003, 0.849, 0.146, 0.005, 0.116, 0.031, 0.853, 0.917, 0.082, 0.001, 0.919, 0.005, 0.075)
population_priors_EUR <- c(0.239, 0.237, 0.525, NA, NA, 0.899, 0.099, 0.002, 0.161, 0.012, 0.827, 0.913, 0.087, NA, 0.799, 0.01, 0.191)
population_priors_FIN <- c(0.202, 0.232, 0.566, NA, NA, 0.909, 0.091, NA, 0.131, 0.01, 0.859, 0.98, 0.02, NA, 0.788, 0.02, 0.192)

sites_of_interest <- c(29814971,47131885,29812725,47132180,29652851)

results_df <- data.frame(Site = integer(), Genotype = character(), Population = character(), Probability = numeric())
normalized_results_list <- list()#useful per third part


for (site in sites_of_interest) {
  for (population in c("Uniform", "ALL", "EUR", "FIN")) {

    population_priors <- switch(population,
                                "Uniform" = rep(1/length(genotypes), length(genotypes)),
                                "ALL" = population_priors_ALL,
                                "EUR" = population_priors_EUR,
                                "FIN" = population_priors_FIN)

    site_data <- subset(data, V2 == site)

    selected_genotypes <- character(nrow(site_data))
    genotype_df <- data.frame(Genotype = genotypes, Probability = numeric(length(genotypes)))# to store all data 

    for (i in 1:nrow(site_data)) {
      bases <- unlist(strsplit(as.character(site_data[i, 5]), split = ","))
      quals <- as.numeric(unlist(strsplit(as.character(site_data[i, 6]), split = ",")))

      ps <- compute_p(bases, quals, genotypes, population_priors)
      
      # dataframe of most frequent genotypes
      selected_genotypes[i] <- genotypes[which.max(ps)]
      results_df <- rbind(results_df, data.frame(Site = site, Genotype = selected_genotypes[i], Population = population, Probability = ps[which.max(ps)]))
        
      genotype_df$Probability <- ps
      #normalize
      total_prob <- sum(ps)
      normalized_ps <- ps / total_prob
        
      genotype_df$Probability <- normalized_ps
      normalized_results_list[[paste(site, population, sep = "_")]] <- genotype_df
    }
  }
}

# Print or further analyze the results_df data frame
print(results_df)
normalized_results_list$'29812725_Uniform'

       Site Genotype Population  Probability
1  29814971       GG    Uniform 1.422834e-59
2  29814971       GG        ALL 1.303280e-36
3  29814971       GA        EUR 2.387305e-38
4  29814971       GA        FIN 8.180746e-37
5  47131885       CT    Uniform 5.253407e-58
6  47131885       CA        ALL 8.261060e-32
7  47131885       CA        EUR 1.024528e-30
8  47131885       CA        FIN 1.666895e-30
9  29812725       CT    Uniform 1.277620e-74
10 29812725       CT        ALL 4.641229e-67
11 29812725       AT        EUR 3.169607e-66
12 29812725       AT        FIN 2.407103e-70
13 47132180       CC    Uniform 2.171924e-86
14 47132180       CA        ALL 1.679633e-41
15 47132180       CA        EUR 8.709686e-40
16 47132180       CA        FIN 1.868494e-39
17 29652851       TT    Uniform 4.360278e-58
18 29652851       CT        ALL 2.618790e-55
19 29652851       AT        EUR 1.072916e-61
20 29652851       CT        FIN 3.657215e-65


Genotype,Probability
<chr>,<dbl>
GG,1.530807e-113
GC,2.020641e-101
GA,1.534980e-102
GT,1.648881e-10
CC,2.621026e-102
CA,8.552000e-93
CT,9.186587e-01
AA,5.172977e-104
AT,6.978589e-02


In [61]:

sites_of_interest <- c(29812725, 29652851)
types <- c("Uniform", "ALL")

#data extraction
extracted_dataframes <- list()

for (site in sites_of_interest) {
  for (type in types) {
    key <- paste(site, type, sep = "_") 

    if (key %in% names(normalized_results_list)) {
      probabilities <- normalized_results_list[[key]]
      

      extracted_dataframes[[key]] <- data.frame(Genotype = rownames(probabilities), Probability = probabilities$Probability)
    }
  }
}

#plotting
for (df_key in names(extracted_dataframes)) {
  plot_obj <- extracted_dataframes[[df_key]]
  plot_title <- paste("Plot for", df_key)
  
  # Create the plot
  pdf(paste0("plot_", df_key, ".pdf"))  # To save the plot as a PDF file
  plot(plot_obj, type = "h", lwd = 20, col = "purple",
       xlab = "Genotypes", ylab = "Prob.", main = plot_title)
  dev.off()  # Close the PDF device
}